In [1]:
#importing all the libraries

import os
import numpy as np
import pandas as pd
import torch
import glob
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [2]:
class SkinCancerDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.annotation = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotation)

    def __getitem__(self, index):
        img_id = self.annotation.iloc[index, 1]#ImageID on column 2
        img_name = os.path.join(self.root_dir, img_id + '.jpg')
        image = Image.open(img_name).convert('RGB')

        y_label = torch.tensor(int(self.annotation.iloc[index]['dx']))

        if self.transform:
            image = self.transform(image)

        return image, y_label

In [3]:
meta = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")
unique_lesion_ids = meta['lesion_id'].unique()

train_ids, temp_ids = train_test_split(unique_lesion_ids, test_size=0.3)
test_ids, validation_ids = train_test_split(temp_ids, test_size=0.5)

train_meta = meta[meta['lesion_id'].isin(train_ids)]
test_meta = meta[meta['lesion_id'].isin(test_ids)]
validation_meta = meta[meta['lesion_id'].isin(validation_ids)]

print(f"train ({100 * len(train_meta) / (len(train_meta) + len(test_meta) + len(validation_meta))}%):\n{train_meta.head()}\n")
print(f"test ({100 * len(test_meta) / (len(train_meta) + len(test_meta) + len(validation_meta))}%):\n{test_meta.head()}\n")
print(f"validation ({100 * len(validation_meta) / (len(train_meta) + len(test_meta) + len(validation_meta))}%):\n{validation_meta.head()}\n")


train (69.82526210683974%):
      lesion_id      image_id   dx dx_type   age     sex localization
2   HAM_0002730  ISIC_0026769  bkl   histo  80.0    male        scalp
3   HAM_0002730  ISIC_0025661  bkl   histo  80.0    male        scalp
8   HAM_0005132  ISIC_0025837  bkl   histo  70.0  female         back
9   HAM_0005132  ISIC_0025209  bkl   histo  70.0  female         back
11  HAM_0004234  ISIC_0029396  bkl   histo  85.0  female        chest

test (14.947578632051922%):
      lesion_id      image_id   dx dx_type   age   sex     localization
4   HAM_0001466  ISIC_0031633  bkl   histo  75.0  male              ear
5   HAM_0001466  ISIC_0027850  bkl   histo  75.0  male              ear
6   HAM_0002761  ISIC_0029176  bkl   histo  60.0  male             face
7   HAM_0002761  ISIC_0029068  bkl   histo  60.0  male             face
16  HAM_0001601  ISIC_0025915  bkl   histo  75.0  male  upper extremity

validation (15.227159261108337%):
      lesion_id      image_id   dx dx_type   age     sex

In [4]:
from torchvision import transforms

# Training transforms (Randomness added)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),       
    transforms.RandomHorizontalFlip(),   
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),       
    transforms.ToTensor(),               
    transforms.Normalize(                # Standardizes to ImageNet distribution
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Validation transforms (No Randomness, just resizing)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
def get_device():
    if torch.cuda.is_available():
        print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
        return torch.device("cuda")
    
    elif torch.backends.mps.is_available():
        print("✅ Apple Silicon GPU Detected")
        return torch.device("mps")
    
    else:
        print("⚠️ No GPU detected. Training will be slow.")
        return torch.device("cpu")


device = get_device()

⚠️ No GPU detected. Training will be slow.


In [6]:
class SkinCancerCNN(nn.Module):
    def __init__(self):
      super().__init__()
    
      self.pool = nn.MaxPool2d(kernel_size=2,stride=2)

      # First block
      self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
      self.bn1 = nn.BatchNorm2d(32)
    
      #second block
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      self.bn2 = nn.BatchNorm2d(64)
    
        
      #third block
      self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
      self.bn3 = nn.BatchNorm2d(128)
    
        
      #fourth block
      self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
      self.bn4 = nn.BatchNorm2d(256)
    
      self.flatten_size = 256 * 14 * 14
      
      # First fully connected layer
      self.fc1 = nn.Linear(self.flatten_size, 128)
      self.dropout = nn.Dropout(0.5)
      # Second fully connected layer that outputs our 10 labels
      self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        # Apply Block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Apply Block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Apply Block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Apply Block 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        #flatten the data
        x = x.view(-1, self.flatten_size)
        #FC layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

skinCancerCNN = SkinCancerCNN()
print(skinCancerCNN)

SkinCancerCNN(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)


In [7]:
labels = meta['dx'].unique()
labels, counts = np.unique(meta['dx'], return_counts=True)
weights = 1.0 / counts 
weights = weights / weights.sum() * len(counts)
weights = torch.tensor(weights, dtype=torch.float)

weights = weights.to(device)

print(labels, counts, weights)

criterion = nn.CrossEntropyLoss(weight=weights)

['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc'] [ 327  514 1099  115 1113 6705  142] tensor([0.9431, 0.6000, 0.2806, 2.6816, 0.2771, 0.0460, 2.1717])
